In [254]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pymongo as pm
import pandas as pd
import collections
import numpy as np
from datetime import datetime
import urllib2
from bokeh.io import curdoc, output_file, show, vform, output_notebook
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select, MultiSelect, CheckboxButtonGroup
from bokeh.plotting import figure

### General Bokeh Workflow:

- Prepare some data (in this case plain python lists).
- Tell Bokeh where to generate output (in this case using output_file(), with the filename "lines.html").
- Call figure() to create a plot with some overall options like title, tools and axes labels.
- Add renderers (in this case, Figure.line) for our data, with visual customizations like colors, legends and widths to the plot.
- Ask Bokeh to show() or save() the results.

In [5]:
mongo_uri = "mongodb://ec2-52-38-154-245.us-west-2.compute.amazonaws.com:27017"
client = pm.MongoClient(mongo_uri)
db = client['prod']

In [110]:
output_notebook()

Loading BokehJS ...

In [9]:
def lru_cache():
    def dec(f):
        def _(*args, **kws):
            return f(*args, **kws)
        return _
    return dec

In [99]:
speaker_queries = pd.DataFrame(list(db.speakers.find({})))
names = sorted(list(name.encode('utf-8','ignore') for name in speaker_queries["name"].unique()))
states = sorted(list(state.encode('utf-8','ignore') for state in speaker_queries["state"].unique()))
years = sorted(list(set(list(str(date.year) for date in speaker_queries[::2]['year']))))

19
12
12


In [223]:
party = CheckboxButtonGroup(labels=['Republican','Democrat','Independent'], active=[0,1,2])
name = CheckboxButtonGroup(labels=names, active=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18])
prior_experience = CheckboxButtonGroup(labels=["Incumbent", "Challenger"], active=[0,1])
election_result = CheckboxButtonGroup(labels=["Win","Lose"], active =[0,1])
state = CheckboxButtonGroup(labels=states, active=[0,1,2,3,4,5,6,7,8,9,10,11])
year = CheckboxButtonGroup(labels=years, active=[0,1,2,3,4,5,6,7,8,9,10,11])

In [219]:
show(party)

In [258]:
def speaker_filter(party_arr, prior_experience_arr, election_result_arr, name_arr, state_arr, date_max, date_min):
    query = {}
    query['party'] = {'$in': party_arr}
    query['incumbent'] = {'$in': prior_experience_arr}
    query['election_result'] = {'$in': election_result_arr}
    query['name'] = {'$in': name_arr}
    query['state'] = {'$in': state_arr}
    query['year'] = {'$lt': date_max, '$gt': date_min}
    mongo_data = list(db.speakers.find(query, {'speaker_id':1}))
    ret_list = []
    for val in mongo_data:
        ret_list.append(val['speaker_id'].encode('utf-8','ignore'))
    return ret_list

In [255]:
import collections
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [285]:
def update_performance_data(selected=None):
    party_arr = [party.labels[i] for i in party.active]
    prior_experience_arr = [True if prior_experience.labels[i] == "Incumbent" else False for i in prior_experience.active]
    election_result_arr = [election_result.labels[i].lower() for i in election_result.active]
    name_arr = [name.labels[i] for i in name.active]
    state_arr = [state.labels[i] for i in state.active]
    date_max = datetime((max([int(year.labels[i]) for i in year.active]) + 1),1,1)
    date_min = datetime((min([int(year.labels[i]) for i in year.active]) - 1),12,31)
    speaker_list = speaker_filter(party_arr, 
                                  prior_experience_arr, 
                                  election_result_arr, 
                                  name_arr, 
                                  state_arr, 
                                  date_max, 
                                  date_min)
    performance_data = pd.DataFrame([flatten(nested_dict) for nested_dict in list(db.performance.find({'$or':[{'speakers.republican.speaker_id':{'$in': speaker_list}}, 
                                                                     {'speakers.democrat.speaker_id':{'$in': speaker_list}}, 
                                                                     {'speakers.independent.speaker_id':{'$in': speaker_list}}]}))]).drop('_id',1)
    performance_data['dem_approval_stock_ratio'] = (performance_data['approval_rates_post_democrat'] - performance_data['approval_rates_pre_democrat'])/performance_data['percent_change_sp500']
    performance_data['rep_approval_stock_ratio'] = (performance_data['approval_rates_post_republican'] - performance_data['approval_rates_pre_republican'])/performance_data['percent_change_sp500']

    return performance_data

In [286]:
def update_debate_data(selected=None):
    party_arr = [party.labels[i] for i in party.active]
    prior_experience_arr = [True if prior_experience.labels[i] == "Incumbent" else False for i in prior_experience.active]
    election_result_arr = [election_result.labels[i].lower() for i in election_result.active]
    name_arr = [name.labels[i] for i in name.active]
    state_arr = [state.labels[i] for i in state.active]
    date_max = datetime((max([int(year.labels[i]) for i in year.active]) + 1),1,1)
    date_min = datetime((min([int(year.labels[i]) for i in year.active]) - 1),12,31)
    speaker_list = speaker_filter(party_arr, 
                                  prior_experience_arr, 
                                  election_result_arr, 
                                  name_arr, 
                                  state_arr, 
                                  date_max, 
                                  date_min)
    debate_data = pd.DataFrame([flatten(nested_dict) for nested_dict in (db.debates.find({'$or':[{'speakers.republican.speaker_id':{'$in': speaker_list}}, 
                                                                     {'speakers.democrat.speaker_id':{'$in': speaker_list}}, 
                                                                     {'speakers.independent.speaker_id':{'$in': speaker_list}}]}))]).drop('_id',1)
    return debate_data

In [287]:
performance = update_performance_data()

In [274]:
for i in performance.columns:
    print i +'=[],'

approval_rates_post_democrat=[],
approval_rates_post_republican=[],
approval_rates_pre_democrat=[],
approval_rates_pre_republican=[],
date=[],
debate_id=[],
percent_change_sp500=[],
speakers_democrat_election_result=[],
speakers_democrat_incumbent=[],
speakers_democrat_name=[],
speakers_democrat_speaker_id=[],
speakers_democrat_state=[],
speakers_democrat_year=[],
speakers_independent_election_result=[],
speakers_independent_incumbent=[],
speakers_independent_name=[],
speakers_independent_speaker_id=[],
speakers_independent_state=[],
speakers_independent_year=[],
speakers_republican_election_result=[],
speakers_republican_incumbent=[],
speakers_republican_name=[],
speakers_republican_speaker_id=[],
speakers_republican_state=[],
speakers_republican_year=[],


### Plots

In [290]:
performance_source = ColumnDataSource(data=dict(approval_rates_post_democrat=[],
                                                approval_rates_post_republican=[],
                                                approval_rates_pre_democrat=[],
                                                approval_rates_pre_republican=[],
                                                date=[],
                                                debate_id=[],
                                                percent_change_sp500=[],
                                                speakers_democrat_election_result=[],
                                                speakers_democrat_incumbent=[],
                                                speakers_democrat_name=[],
                                                speakers_democrat_speaker_id=[],
                                                speakers_democrat_state=[],
                                                speakers_democrat_year=[],
                                                speakers_independent_election_result=[],
                                                speakers_independent_incumbent=[],
                                                speakers_independent_name=[],
                                                speakers_independent_speaker_id=[],
                                                speakers_independent_state=[],
                                                speakers_independent_year=[],
                                                speakers_republican_election_result=[],
                                                speakers_republican_incumbent=[],
                                                speakers_republican_name=[],
                                                speakers_republican_speaker_id=[],
                                                speakers_republican_state=[],
                                                speakers_republican_year=[],
                                                dem_approval_stock_ratio=[],
                                                rep_approval_stock_ratio=[]
                                               ))

performance_static_source = ColumnDataSource(data=dict(approval_rates_post_democrat=[],
                                                approval_rates_post_republican=[],
                                                approval_rates_pre_democrat=[],
                                                approval_rates_pre_republican=[],
                                                date=[],
                                                debate_id=[],
                                                percent_change_sp500=[],
                                                speakers_democrat_election_result=[],
                                                speakers_democrat_incumbent=[],
                                                speakers_democrat_name=[],
                                                speakers_democrat_speaker_id=[],
                                                speakers_democrat_state=[],
                                                speakers_democrat_year=[],
                                                speakers_independent_election_result=[],
                                                speakers_independent_incumbent=[],
                                                speakers_independent_name=[],
                                                speakers_independent_speaker_id=[],
                                                speakers_independent_state=[],
                                                speakers_independent_year=[],
                                                speakers_republican_election_result=[],
                                                speakers_republican_incumbent=[],
                                                speakers_republican_name=[],
                                                speakers_republican_speaker_id=[],
                                                speakers_republican_state=[],
                                                speakers_republican_year=[],
                                                dem_approval_stock_ratio=[],
                                                rep_approval_stock_ratio=[]
                                                      ))
performance_plot = figure(plot_width=500, 
                          plot_height=250, 
                          tools='pan')
performance_plot.scatter('date', 'dem_approval_stock_ratio', source=performance_source, color='blue')
performance_plot.scatter('date', 'rep_approval_stock_ratio', source=performance_source, color='red')

GlyphRenderer(id='ab7b0c5b-6485-4f24-97c1-a89b79aa2a13', ...)

In [288]:
for val in performance.columns:
    print "\'" + val +"\',"

'approval_rates_post_democrat',
'approval_rates_post_republican',
'approval_rates_pre_democrat',
'approval_rates_pre_republican',
'date',
'debate_id',
'percent_change_sp500',
'speakers_democrat_election_result',
'speakers_democrat_incumbent',
'speakers_democrat_name',
'speakers_democrat_speaker_id',
'speakers_democrat_state',
'speakers_democrat_year',
'speakers_independent_election_result',
'speakers_independent_incumbent',
'speakers_independent_name',
'speakers_independent_speaker_id',
'speakers_independent_state',
'speakers_independent_year',
'speakers_republican_election_result',
'speakers_republican_incumbent',
'speakers_republican_name',
'speakers_republican_speaker_id',
'speakers_republican_state',
'speakers_republican_year',
'dem_approval_stock_ratio',
'rep_approval_stock_ratio',


In [291]:
performance_source.data = performance_source.from_df(performance[['approval_rates_post_democrat',
                                                                  'approval_rates_post_republican',
                                                                  'approval_rates_pre_democrat',
                                                                  'approval_rates_pre_republican',
                                                                  'date',
                                                                  'debate_id',
                                                                  'percent_change_sp500',
                                                                  'speakers_democrat_election_result',
                                                                  'speakers_democrat_incumbent',
                                                                  'speakers_democrat_name',
                                                                  'speakers_democrat_speaker_id',
                                                                  'speakers_democrat_state',
                                                                  'speakers_democrat_year',
                                                                  'speakers_independent_election_result',
                                                                  'speakers_independent_incumbent',
                                                                  'speakers_independent_name',
                                                                  'speakers_independent_speaker_id',
                                                                  'speakers_independent_state',
                                                                  'speakers_independent_year',
                                                                  'speakers_republican_election_result',
                                                                  'speakers_republican_incumbent',
                                                                  'speakers_republican_name',
                                                                  'speakers_republican_speaker_id',
                                                                  'speakers_republican_state',
                                                                  'speakers_republican_year',
                                                                  'dem_approval_stock_ratio',
                                                                  'rep_approval_stock_ratio']])

In [292]:
show(performance_plot)

ValueError: NaTType does not support timetuple